In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
import sklearn 

In [4]:
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [5]:
train = pd.read_csv("Predicting-House-Prices-In-Bengaluru-Train-Data.csv")
test = pd.read_csv("Predicting-House-Prices-In-Bengaluru-Test-Data.csv")

In [6]:
print("train data set shape: ",train.shape)
print("test data set shape: ",test.shape)

train data set shape:  (13320, 9)
test data set shape:  (1480, 9)


In [16]:
13320+1480

14800

In [9]:
print("train head:")
train.head()

train head:


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [10]:
print("test head:")
test.head()

test head:


,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,Ready To Move,Brookefield,2 BHK,Roeekbl,1225,2.0,2.0,NaN
1,Plot Area,Ready To Move,Akshaya Nagar,9 Bedroom,NaN,2400,9.0,2.0,NaN
2,Plot Area,18-Apr,Hennur Road,4 Bedroom,Saandtt,1650,5.0,2.0,NaN
3,Super built-up Area,Ready To Move,Kodichikkanahalli,3 BHK,Winerri,1322,3.0,1.0,NaN
4,Super built-up Area,Ready To Move,Konanakunte,2 BHK,AmageSa,1161,2.0,1.0,NaN


In [13]:
dataframes = [train,test]
data = pd.concat(dataframes)

AttributeError: module 'pandas' has no attribute 'append'

In [14]:
data.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,Super built-up Area,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,Plot Area,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,Built-up Area,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,Super built-up Area,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,Super built-up Area,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [15]:
data.shape

(14800, 9)

In [17]:
data.area_type.value_counts()

Super built-up  Area    9736
Built-up  Area          2688
Plot  Area              2279
Carpet  Area              97
Name: area_type, dtype: int64

In [18]:
data["area_type"] = data.area_type.map({"Super built-up  Area" : 1, "Built-up  Area" : 2 , "Plot  Area" : 3, "Carpet  Area": 4})

In [20]:
data.head(5)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,19-Dec,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,3,Ready To Move,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,2,Ready To Move,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,1,Ready To Move,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,1,Ready To Move,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [21]:
def replace_availabilty(type_of_availability):
    if type_of_availability == 'Ready To Move':
        return 0
    elif type_of_availability == 'Immediate Possession':
        return 1
    else:
        return 2

In [22]:
data["availability"] = data.availability.apply(replace_availabilty)

In [23]:
data.head(5)

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,2,Electronic City Phase II,2 BHK,Coomee,1056,2.0,1.0,39.07
1,3,0,Chikka Tirupathi,4 Bedroom,Theanmp,2600,5.0,3.0,120.00
2,2,0,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
3,1,0,Lingadheeranahalli,3 BHK,Soiewre,1521,3.0,1.0,95.00
4,1,0,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00


In [24]:
data.isna().sum()

area_type          0
availability       0
location           1
size              18
society         6128
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [26]:
data[data["location"].isnull()]

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
568,1,0,NaN,3 BHK,Grare S,1600,3.0,2.0,86.0


In [32]:
data2 = data[data['location'].notnull()]
data2.shape

(14799, 9)

In [35]:
data2.isna().sum()

area_type          0
availability       0
location           0
size              18
society         6128
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [37]:
data2[data2["size"].isnull()].head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
579,3,1,Sarjapur Road,NaN,Asiss B,1200 - 2400,NaN,NaN,34.185
1775,3,1,IVC Road,NaN,Orana N,2000 - 5634,NaN,NaN,124.000
2264,3,1,Banashankari,NaN,NaN,2400,NaN,NaN,460.000
2809,3,1,Sarjapur Road,NaN,AsdiaAr,1200 - 2400,NaN,NaN,28.785
2862,3,1,Devanahalli,NaN,Ajleyor,1500 - 2400,NaN,NaN,46.800


In [38]:
data2[data2["society"].isnull()].head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
2,2,0,Uttarahalli,3 BHK,NaN,1440,2.0,3.0,62.00
4,1,0,Kothanur,2 BHK,NaN,1200,2.0,1.0,51.00
8,1,0,Marathahalli,3 BHK,NaN,1310,3.0,1.0,63.25
9,3,0,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.00
10,1,2,Whitefield,3 BHK,NaN,1800,2.0,2.0,70.00


In [39]:
data2[data2["total_sqft"].isnull()]

,area_type,availability,location,size,society,total_sqft,bath,balcony,price


In [40]:
data2[data2["bath"].isnull()].head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
56,2,2,Devanahalli,4 Bedroom,BrereAt,3010 - 3410,NaN,NaN,192.000
81,2,2,Hennur Road,4 Bedroom,Gollela,2957 - 3450,NaN,NaN,224.500
224,1,2,Devanahalli,3 BHK,Jurdsig,1520 - 1740,NaN,NaN,74.820
344,1,2,Kanakpura Road,1 BHK,PrarePa,525,NaN,NaN,21.530
579,3,1,Sarjapur Road,NaN,Asiss B,1200 - 2400,NaN,NaN,34.185


In [41]:
data2[data2["balcony"].isnull()].head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
6,1,2,Old Airport Road,4 BHK,Jaades,2732,4.0,NaN,204.0
7,1,0,Rajaji Nagar,4 BHK,Brway G,3300,4.0,NaN,600.0
9,3,0,Gandhi Bazar,6 Bedroom,NaN,1020,6.0,NaN,370.0
34,2,0,Kasturi Nagar,3 BHK,Kantsce,1925,3.0,NaN,125.0
40,2,0,Murugeshpalya,2 BHK,Gentson,1296,2.0,NaN,81.0


In [42]:
data2.dtypes

area_type         int64
availability      int64
location         object
size             object
society          object
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [43]:
(data2["society"].isna().sum()/data2.shape[0]) *  100

41.40820325697682

In [44]:
data2["society"] = data2["society"].fillna("Not_mentioned")

In [45]:
(data2["society"].isna().sum()/data2.shape[0]) *  100

0.0

In [46]:
data2.society.value_counts().head()

Not_mentioned    6128
GrrvaGr            92
PrarePa            78
Prtates            64
Bhmesy             63
Name: society, dtype: int64

In [47]:
train.society.value_counts().head()

GrrvaGr    80
PrarePa    76
Prtates    59
Sryalan    59
GMown E    56
Name: society, dtype: int64

In [48]:
society_labels = LabelEncoder()
society_labels.fit(data2["society"])
data2["society"] = society_labels.transform(data2["society"])
data2.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,2,Electronic City Phase II,2 BHK,487,1056,2.0,1.0,39.07
1,3,0,Chikka Tirupathi,4 Bedroom,2569,2600,5.0,3.0,120.00
2,2,0,Uttarahalli,3 BHK,1505,1440,2.0,3.0,62.00
3,1,0,Lingadheeranahalli,3 BHK,2303,1521,3.0,1.0,95.00
4,1,0,Kothanur,2 BHK,1505,1200,2.0,1.0,51.00


In [51]:
(data2["size"].isna().sum()/data2.shape[0]) *  100

0.1216298398540442

In [52]:
data2["size"] = data2["size"].fillna("Not_mentioned")

In [53]:
(data2["size"].isna().sum()/data2.shape[0]) *  100

0.0

In [54]:
size_labels = LabelEncoder()
size_labels.fit(data2["size"])
data2["size"] = size_labels.transform(data2["size"])
data2.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,2,Electronic City Phase II,14,487,1056,2.0,1.0,39.07
1,3,0,Chikka Tirupathi,20,2569,2600,5.0,3.0,120.00
2,2,0,Uttarahalli,17,1505,1440,2.0,3.0,62.00
3,1,0,Lingadheeranahalli,17,2303,1521,3.0,1.0,95.00
4,1,0,Kothanur,14,1505,1200,2.0,1.0,51.00


In [55]:
(data2["location"].isna().sum()/data2.shape[0]) *  100

0.0

In [56]:
location_labels = LabelEncoder()
location_labels.fit(data2["location"])
data2["location"] = location_labels.transform(data2["location"])
data2.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony,price
0,1,2,430,14,487,1056,2.0,1.0,39.07
1,3,0,325,20,2569,2600,5.0,3.0,120.00
2,2,0,1219,17,1505,1440,2.0,3.0,62.00
3,1,0,778,17,2303,1521,3.0,1.0,95.00
4,1,0,736,14,1505,1200,2.0,1.0,51.00


In [57]:
data2.isna().sum()

area_type          0
availability       0
location           0
size               0
society            0
total_sqft         0
bath              80
balcony          678
price           1480
dtype: int64

In [58]:
data2.dtypes

area_type         int64
availability      int64
location          int32
size              int32
society           int32
total_sqft       object
bath            float64
balcony         float64
price           float64
dtype: object

In [59]:
column_balcony = data2.groupby('location')['balcony'].transform(lambda x: x.fillna(x.mean()))
column_balcony = column_balcony.fillna(column_balcony.mean())

In [60]:
column_bath = data2.groupby('location')['bath'].transform(lambda x: x.fillna(x.mean()))
column_bath = column_bath.fillna(column_bath.mean())

In [61]:
column_balcony.isna().sum()


0

In [62]:
column_bath.isna().sum()


0

In [63]:
data2['balcony'] = column_balcony

In [64]:
data2['bath'] = column_bath

In [65]:
data2.isna().sum()

area_type          0
availability       0
location           0
size               0
society            0
total_sqft         0
bath               0
balcony            0
price           1480
dtype: int64

In [67]:
data2.shape

(14799, 9)

In [68]:
training_data = data2[data2['price'].notnull()]

In [69]:
training_data.shape

(13319, 9)

In [70]:
train.shape

(13320, 9)

In [72]:
testing_data = data2[data2['price'].isnull()]

In [73]:
testing_data.shape

(1480, 9)

In [74]:
testing_data = testing_data.drop(columns='price')
testing_data.shape


(1480, 8)

In [76]:
!pip install xgboost
import xgboost as xgb

In [81]:
!pip install sklearn

  Stored in directory: C:\Users\Admin\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [83]:
from sklearn.model_selection import GridSearchCV

In [86]:
columns = training_data.columns
X_train = training_data[columns[:-1]]
y_train = training_data[columns[-1]]

In [87]:
X_test = testing_data
X_test.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony
0,1,0,284,14,1876,1225,2.0,2.0
1,3,0,103,31,1505,2400,9.0,2.0
2,3,2,534,20,2030,1650,5.0,2.0
3,1,0,721,17,2817,1322,3.0,1.0
4,1,0,727,14,135,1161,2.0,1.0


In [89]:
X_train.head()

,area_type,availability,location,size,society,total_sqft,bath,balcony
0,1,2,430,14,487,1056,2.0,1.0
1,3,0,325,20,2569,2600,5.0,3.0
2,2,0,1219,17,1505,1440,2.0,3.0
3,1,0,778,17,2303,1521,3.0,1.0
4,1,0,736,14,1505,1200,2.0,1.0


In [90]:
y_train.head()

0     39.07
1    120.00
2     62.00
3     95.00
4     51.00
Name: price, dtype: float64

In [91]:
X_train.dtypes

area_type         int64
availability      int64
location          int32
size              int32
society           int32
total_sqft       object
bath            float64
balcony         float64
dtype: object

In [92]:
obj_cols = X_train.columns[X_train.dtypes.eq(object)]

In [93]:
obj_cols

Index(['total_sqft'], dtype='object')

In [95]:
X_train[obj_cols] =X_train[obj_cols].apply(pd.to_numeric, errors='coerce')

In [96]:
X_train.dtypes

area_type         int64
availability      int64
location          int32
size              int32
society           int32
total_sqft      float64
bath            float64
balcony         float64
dtype: object

In [97]:
y_train.dtypes

dtype('float64')

In [99]:
from sklearn.preprocessing import Imputer

In [101]:
my_imputer = Imputer()
train_X = my_imputer.fit_transform(X_train)

In [103]:
import xgboost 

In [104]:
best_xgb_model = xgboost.XGBRegressor(colsample_bytree=0.4,
                 gamma=0,                 
                 learning_rate=0.07,
                 max_depth=3,
                 min_child_weight=1.5,
                 n_estimators=10000,                                                                    
                 reg_alpha=0.75,
                 reg_lambda=0.45,
                 subsample=0.6,
                 seed=42)
best_xgb_model.fit(train_X,y_train)

[22:11:25] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.4, gamma=0,
       importance_type='gain', learning_rate=0.07, max_delta_step=0,
       max_depth=3, min_child_weight=1.5, missing=None, n_estimators=10000,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0.75, reg_lambda=0.45, scale_pos_weight=1, seed=42,
       silent=None, subsample=0.6, verbosity=1)

In [106]:
X_test.dtypes

area_type         int64
availability      int64
location          int32
size              int32
society           int32
total_sqft       object
bath            float64
balcony         float64
dtype: object

In [107]:
X_test[obj_cols] =X_test[obj_cols].apply(pd.to_numeric, errors='coerce')

In [108]:
X_test.dtypes

area_type         int64
availability      int64
location          int32
size              int32
society           int32
total_sqft      float64
bath            float64
balcony         float64
dtype: object

In [109]:
test_X = my_imputer.fit_transform(X_test)

In [110]:
submission = best_xgb_model.predict(test_X)

In [118]:
df = pd.DataFrame({'price':submission})

In [119]:
df.shape

(1480, 1)

In [120]:
df.head()

,price
0,75.449020
1,344.248199
2,354.301880
3,57.218498
4,51.249111


In [121]:
df.to_csv('xgboost1.csv')